<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-Up" data-toc-modified-id="Set-Up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set Up</a></span></li><li><span><a href="#Prepare-training-data" data-toc-modified-id="Prepare-training-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare training data</a></span></li><li><span><a href="#(Conv-=>-Pool)-*-2-Model" data-toc-modified-id="(Conv-=>-Pool)-*-2-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><code>(Conv =&gt; Pool) * 2</code> Model</a></span></li><li><span><a href="#Conv-=>-Conv-=>-Pool-Model" data-toc-modified-id="Conv-=>-Conv-=>-Pool-Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><code>Conv =&gt; Conv =&gt; Pool</code> Model</a></span></li><li><span><a href="#Test-on-MNIST" data-toc-modified-id="Test-on-MNIST-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Test on MNIST</a></span></li><li><span><a href="#Image-Input-Recognition-via-cv2" data-toc-modified-id="Image-Input-Recognition-via-cv2-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Image Input Recognition via cv2</a></span></li><li><span><a href="#Pygame-Interface" data-toc-modified-id="Pygame-Interface-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Pygame Interface</a></span></li></ul></div>

# Set Up

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import pandas as pd
import os

# Define global variables
INPUT_SHAPE = (28,28,1)
NUMBER_OF_CLASSES = 13
LABEL_NAMES = ['0','1','2','3','4','5','6','7','8','9','-','+','*']

# Prepare training data

In [2]:
import pickle
df_train=pd.read_csv('train_final.csv',index_col=False)

In [3]:
labels=df_train[['784']]
df_train.drop(df_train.columns[[784]],axis=1,inplace=True)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Output vector
y_train=np.array(labels).reshape(-1,)

# Input vector
l=[]
for i in range(47504):
    l.append(np.array(df_train[i:i+1]).reshape(28,28,1))
X_train = np.array(l)

# Cross check dimensions
print(X_train.shape, y_train.shape)

(47504, 28, 28, 1) (47504,)


# `(Conv => Pool) * 2` Model

Reference: https://medium.com/@vipul.gupta73921/handwritten-equation-solver-using-convolutional-neural-network-a44acc0bd9f8

Credit: Vipul Gupta

In [5]:
# Define CNN
class vipul:
    @staticmethod
    def build():
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Conv2D(30, (5, 5), input_shape=INPUT_SHAPE, activation='relu'))
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Conv2D(15, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(128, activation='relu'))
        model.add(tf.keras.layers.Dense(50, activation='relu'))
        model.add(tf.keras.layers.Dense(NUMBER_OF_CLASSES, activation='softmax'))
        return model

# Compile model

vipul_model = vipul.build()
    
vipul_model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

vipul_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 30)        780       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 15)        4065      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 15)          0         
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 15)          0         
_________________________________________________________________
flatten (Flatten)            (None, 375)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [6]:
# Train the model (if weight file not exists)
if os.path.isfile('vipul_final.h5') == False:
    # Set random seed for shuffling
    np.random.seed(7)

    # Train model
    vipul_model.fit(X_train, y_train, epochs=10, batch_size=200,shuffle=True,verbose=1)

    # Save model
    with open("vipul_final.json", "w") as json_file:
        json_file.write(vipul_model.to_json())
        
    # Save weights
    vipul_model.save_weights("vipul_final.h5")
else:
    # Load pre-trained weight
    vipul_model.load_weights('vipul_final.h5')

# `Conv => Conv => Pool` Model

Reference: https://medium.com/@ayrusreev/real-time-digit-recognition-using-keras-5f333c0163e2

Credit: Suryaveer Singh

In [7]:
# Define CNN
class Suryaveer:
    @staticmethod
    def build():
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=INPUT_SHAPE))
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(128, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(tf.keras.layers.Dense(NUMBER_OF_CLASSES, activation='softmax'))
        return model

# Build & Compile
suryaveer_model = Suryaveer.build()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

suryaveer_model.compile(loss='sparse_categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'],
                 callbacks=[reduce_lr])

# Model Summary
suryaveer_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [8]:
# Train the model (if weight file not exists)
if os.path.isfile('suryaveer_final.h5') == False:
    # Set random seed for shuffling
    np.random.seed(7)

    # Train model
    suryaveer_model.fit(X_train, y_train, epochs=25, batch_size=200,shuffle=True,verbose=1)

    # Save model
    with open("suryaveer_final.json", "w") as json_file:
        json_file.write(suryaveer_model.to_json())
        
    # Save weights
    suryaveer_model.save_weights("suryaveer_final.h5")
else:
    # Load pre-trained weight
    suryaveer_model.load_weights('suryaveer_final.h5')

# Test on MNIST

In [9]:
# Import MNIST test
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshaping 
X_test = tf.cast(X_test.reshape(-1,28,28,1), tf.float32)

# Cross checking shape of input and output
print(X_test.shape, y_test.shape)

(10000, 28, 28, 1) (10000,)


In [10]:
from sklearn.metrics import classification_report, accuracy_score

# Vipul Model
print('Vipul Model Testing Report')
preds = vipul_model.predict(X_test)
print('Accuracy Score:',accuracy_score(y_test, preds.argmax(axis=1)))
print('-'*60)
print(classification_report(y_test, preds.argmax(axis=1), labels = range(13), target_names=LABEL_NAMES))

print('='*60)

# Suryaveer Model
print('Suryaveer Model Testing Report')
preds = suryaveer_model.predict(X_test)
print('Accuracy Score:',accuracy_score(y_test, preds.argmax(axis=1)))
print('-'*60)
print(classification_report(y_test, preds.argmax(axis=1), labels = range(13), target_names=LABEL_NAMES))

Vipul Model Testing Report
Accuracy Score: 0.1828
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.04      0.07       980
           1       0.72      0.42      0.53      1135
           2       1.00      0.02      0.03      1032
           3       0.30      0.24      0.26      1010
           4       0.15      0.90      0.26       982
           5       0.47      0.19      0.27       892
           6       0.14      0.00      0.01       958
           7       0.24      0.00      0.01      1028
           8       0.50      0.01      0.01       974
           9       0.00      0.00      0.00      1009
           -       0.00      0.00      0.00         0
           +       0.00      0.00      0.00         0
           *       0.00      0.00      0.00         0

   micro avg       0.18      0.18      0.18     10000
   macro avg       0.35      0.14      0.11     10000
weighted avg       0.46

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Accuracy Score: 0.2145
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.46      0.02      0.04       980
           1       0.37      0.23      0.28      1135
           2       0.74      0.07      0.13      1032
           3       0.63      0.19      0.29      1010
           4       0.14      0.98      0.24       982
           5       0.53      0.62      0.57       892
           6       0.46      0.08      0.13       958
           7       0.06      0.00      0.00      1028
           8       0.29      0.01      0.02       974
           9       0.05      0.00      0.00      1009
           -       0.00      0.00      0.00         0
           +       0.00      0.00      0.00         0
           *       0.00      0.00      0.00         0

    accuracy                           0.21     10000
   macro avg       0.29      0.17      0.13     10000
weighted avg       0.37      0.21      0.17     10

# Image Input Recognition via cv2

In [11]:
import cv2
from scipy import ndimage
import math

In [12]:
def predict_digit(img):
    
    # Change to 'vipul_model' or 'suryaveer_model'
    model = suryaveer_model
    
    test_image = img.reshape(-1,28,28,1)
    test_image = tf.cast(test_image, tf.float32)
    pred = np.argmax(model.predict(test_image))
    return LABEL_NAMES[pred]

def print_predict(pred_str):
    print('-'*40)
    print('Your input is: ', pred_str)
    try:
        print('Result: '+pred_str+' =', eval(pred_str))
    except:
        print('Cannot evaluate input!')
    return

#pitting label
def put_label(t_img,label,x,y):
    font = cv2.FONT_HERSHEY_SIMPLEX
    l_x = int(x) - 10
    l_y = int(y) + 10
    cv2.rectangle(t_img,(l_x,l_y+5),(l_x+35,l_y-35),(0,255,0),-1) 
    cv2.putText(t_img,str(label),(l_x,l_y), font,1.5,(255,0,0),1,cv2.LINE_AA)
    return t_img

# refining each digit
def image_refiner(gray):
    org_size = 22
    img_size = 28
    rows,cols = gray.shape
    
    if rows > cols:
        factor = org_size/rows
        rows = org_size
        cols = int(round(cols*factor))        
    else:
        factor = org_size/cols
        cols = org_size
        rows = int(round(rows*factor))
    gray = cv2.resize(gray, (cols, rows))
    
    #get padding 
    colsPadding = (int(math.ceil((img_size-cols)/2.0)),int(math.floor((img_size-cols)/2.0)))
    rowsPadding = (int(math.ceil((img_size-rows)/2.0)),int(math.floor((img_size-rows)/2.0)))
    
    #apply padding 
    gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant')
    return gray

def get_output_image(path):
  
    img = cv2.imread(path,2)
    img_org =  cv2.imread(path)

    ret,thresh = cv2.threshold(img,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    
    h_list=[]
    c_list = enumerate(contours)
    
    for j,cnt in c_list:
        epsilon = 0.01*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)
        
        hull = cv2.convexHull(cnt)
        k = cv2.isContourConvex(cnt)
        x,y,w,h = cv2.boundingRect(cnt)
        
        # Create a contour list sorted ascending y-axis
        if(hierarchy[0][j][3]!=-1 and w>10 and h>10):
            h_list.append([x,y,w,h,cnt])
    
    # Create a contour list sorted by ascending x-axis
    ziped_list = list(zip(*h_list))
    x_list = list(ziped_list[0])
    dic = dict(zip(x_list,h_list))
    x_list.sort()
    
    # Predicted string of input
    pred_input = ''
    
    for i in range(len(x_list)):
        [x,y,w,h,cnt]=dic[x_list[i]]
        #putting boundary on each digit
        cv2.rectangle(img_org,(x,y),(x+w,y+h),(0,255,0),2)
            
        #cropping each image and process
        roi = img[y:y+h, x:x+w]
        roi = cv2.bitwise_not(roi)
        roi = image_refiner(roi)
        th,fnl = cv2.threshold(roi,127,255,cv2.THRESH_BINARY)
            
        # getting prediction of cropped image
        pred = predict_digit(roi)
        pred_input += pred
            
        # placing label on each digit
        (x,y),radius = cv2.minEnclosingCircle(cnt)
        img_org = put_label(img_org,pred,x,y)
    
    # Print output prediction & evaluation
    print_predict(pred_input)

    return img_org

# Pygame Interface

In [20]:
import pygame

'''
Press 'Enter' to evaluate in stdout
Left click to reset canvas
Press 'Esc' to close
'''

# pre defined colors, pen radius and font color
black = [0, 0, 0]
white = [255, 255, 255]
red = [255, 0, 0]
green = [0, 255, 0]
draw_on = False
last_pos = (0, 0)
color = (255, 128, 0)
radius = 7
font_size = 500

#image size
width = 640
height = 640

# initializing screen
screen = pygame.display.set_mode((width*2, height))
screen.fill(white)
pygame.font.init()

def show_output_image(img):
    surf = pygame.pixelcopy.make_surface(img)
    surf = pygame.transform.rotate(surf, -270)
    surf = pygame.transform.flip(surf, 0, 1)
    screen.blit(surf, (width+2, 0))

def crope(orginal):
    cropped = pygame.Surface((width-5, height-5))
    cropped.blit(orginal, (0, 0), (0, 0, width-5, height-5))
    return cropped

def roundline(srf, color, start, end, radius=1):
    dx = end[0] - start[0]
    dy = end[1] - start[1]
    distance = max(abs(dx), abs(dy))
    for i in range(distance):
        x = int(start[0] + float(i) / distance * dx)
        y = int(start[1] + float(i) / distance * dy)
        pygame.draw.circle(srf, color, (x, y), radius)

def draw_partition_line():
    pygame.draw.line(screen, black, [width, 0], [width,height ], 8)

try:
    while True:
        # get all events
        e = pygame.event.wait()
        draw_partition_line()

        # clear screen after right click
        if(e.type == pygame.MOUSEBUTTONDOWN and e.button == 3):
            screen.fill(white)
            print('='*40)
            print('New Input')

        # quit
        if e.type == pygame.QUIT or (e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE):
            print('='*15+' FINISHED '+'='*15)
            raise StopIteration

        # start drawing after left click
        if(e.type == pygame.MOUSEBUTTONDOWN and e.button != 3):
            color = black
            pygame.draw.circle(screen, color, e.pos, radius)
            draw_on = True

        # stop drawing after releasing left click
        if e.type == pygame.MOUSEBUTTONUP and e.button != 3:
            draw_on = False
            fname = "out.png"
            
            img = crope(screen)
            pygame.image.save(img, fname)
            
        if e.type == pygame.KEYDOWN and e.key == pygame.K_RETURN:
            output_img = get_output_image(fname)
            show_output_image(output_img)

        # start drawing line on screen if draw is true
        if e.type == pygame.MOUSEMOTION:
            if draw_on:
                pygame.draw.circle(screen, color, e.pos, radius)
                roundline(screen, color, e.pos, last_pos, radius)
            last_pos = e.pos

        pygame.display.flip()

except StopIteration:
    pass

pygame.quit()

=============== FINISHED ===============
